## Cross-Encoders

Playground for Cross-Encoders. This is meant to thest the cross-encoder capacity in improving retrieval tasks

## Loading datasets

The following cell loads the datasets by using the natural questions dataset from HuggingFace

In [1]:
%load_ext autoreload
%autoreload 2

# Hugging face datasets helper
from datasets import load_dataset

train_dataset = load_dataset("sentence-transformers/natural-questions", split="train")

print(train_dataset)


print("Query: ", train_dataset[0]["query"])
print("Answer:", train_dataset[0]["answer"])

Dataset({
    features: ['query', 'answer'],
    num_rows: 100231
})
Query:  when did richmond last play in a preliminary final
Answer: Richmond Football Club Richmond began 2017 with 5 straight wins, a feat it had not achieved since 1995. A series of close losses hampered the Tigers throughout the middle of the season, including a 5-point loss to the Western Bulldogs, 2-point loss to Fremantle, and a 3-point loss to the Giants. Richmond ended the season strongly with convincing victories over Fremantle and St Kilda in the final two rounds, elevating the club to 3rd on the ladder. Richmond's first final of the season against the Cats at the MCG attracted a record qualifying final crowd of 95,028; the Tigers won by 51 points. Having advanced to the first preliminary finals for the first time since 2001, Richmond defeated Greater Western Sydney by 36 points in front of a crowd of 94,258 to progress to the Grand Final against Adelaide, their first Grand Final appearance since 1982. The at

In [2]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import mine_hard_negatives

from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.losses import CachedMultipleNegativesRankingLoss


# train_dataset = load_dataset("sentence-transformers/gooaq", split="train").select(range(100_000))
# print(train_dataset)

# # Mine hard negatives using a very efficient embedding model
# embedding_model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
# hard_train_dataset = mine_hard_negatives(
#     train_dataset,
#     embedding_model,
#     num_negatives=5,  # How many negatives per question-answer pair
#     range_min=10,  # Skip the x most similar samples
#     range_max=100,  # Consider only the x most similar samples
#     max_score=0.8,  # Only consider samples with a similarity score of at most x
#     absolute_margin=0.1,  # Similarity between query and negative samples should be x lower than query-positive similarity
#     sampling_strategy="top",  # Randomly sample negatives from the range
#     batch_size=4096,  # Use a batch size of 4096 for the embedding model
#     output_format="labeled-pair",  # The output format is (query, passage, label), as required by BinaryCrossEntropyLoss
#     use_faiss=True,  # Using FAISS is recommended to keep memory usage low (pip install faiss-gpu or pip install faiss-cpu)
# )
# print(hard_train_dataset)
# print(hard_train_dataset[1])



In [ ]:
from sentence_transformers.cross_encoder import CrossEncoderTrainingArguments
from sentence_transformers.training_args import BatchSamplers

model = CrossEncoder("xlm-roberta-base", num_labels=1) # num_labels=1 is for rerankers
loss = CachedMultipleNegativesRankingLoss(model)
train_dataset = load_dataset("sentence-transformers/gooaq", split="train")

args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir="models/reranker-MiniLM-msmarco-v1",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="reranker-MiniLM-msmarco-v1",  # Will be used in W&B if `wandb` is installed
)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
